In [2]:
import pandas as pd
import numpy as np#### Load Packages dir(imlreliability)

from sklearn.preprocessing import scale, normalize
communities_data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/communities/communities.data').to_numpy()
communities_data = np.delete(communities_data, np.arange(5), 1)
    # remove predictors with missing values
communities_data = np.delete(communities_data,
                             np.argwhere((communities_data == '?').sum(0) > 0).reshape(-1), 1)
communities_data = communities_data.astype(float)
x = communities_data[:, :-1]
y = communities_data[:, -1]


### scale and normalize data 
x = normalize(scale(x))
y = (scale(y))
data_reg=(x,y)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
M=len(x[0])
noise='split'
i=0
var=1
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.3,random_state=i*1234+7)
x_train=preprocessing.scale(x_train)
x_test =preprocessing.scale(x_test)
y_train =preprocessing.scale(y_train)
y_test =preprocessing.scale(y_test)  


In [4]:
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense
model = Sequential()
model.add(Dense(M, input_dim=M, activation='relu'))
model.add(Dense(M, input_dim=M, activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train,y_train, epochs=10, batch_size=50)
model_json = model.to_json()


2023-05-16 15:19:47.586474: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-16 15:19:47.586496: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


2023-05-16 15:19:47.831134: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-16 15:19:47.947925: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Epoch 1/10
28/28 [==============================] - 0s 4ms/step - loss: 0.5257
Epoch 2/10
28/28 [==============================] - 0s 3ms/step - loss: 0.3431
Epoch 3/10
28/28 [==============================] - 0s 4ms/step - loss: 0.3205
Epoch 4/10
28/28 [==============================] - 0s 4ms/step - loss: 0.2996
Epoch 5/10
28/28 [==============================] - 0s 4ms/step - loss: 0.2798
Epoch 6/10
28/28 [==============================] - 0s 3ms/step - loss: 0.2613
Epoch 7/10
28/28 [==============================] - 0s 4ms/step - loss: 0.2389
Epoch 8/10
28/28 [==============================] - 0s 3ms/step - loss: 0.2371
Epoch 9/10
28/28 [==============================] - 0s 3ms/step - loss: 0.2161
Epoch 10/10
28/28 [==============================] - 0s 4ms/step - loss: 0.1986


In [5]:
with open("checkdl.json", "w") as json_file:
    json_file.write(model_json)

#                     # serialize weights to HDF5
model.save("checkdl.h5")

saved_model_file="checkdl.h5"


In [6]:
from deepexplain.tensorflow import DeepExplain
for method_name,func_name in [
                              ('IntegratedGradients','intgrad'),
                          ]:
    print("Computing scores for:",method_name)
    scores,ranks={},{}
    scores[method_name],ranks[method_name]=[],[]

    with DeepExplain(session=K.get_session()) as de:  # <-- init DeepExplain context
            # Need to reconstruct the graph in DeepExplain context, using the same weights.
            # With Keras this is very easy:
            # 1. Get the input tensor to the original model
            input_tensor = model.layers[0].input

            # 2. We now target the output of the last dense layer (pre-softmax)
            # To do so, create a new model sharing the same layers untill the last dense (index -2)
            fModel = Model(inputs=input_tensor, outputs = model.layers[-2].output)
            target_tensor = fModel(input_tensor)

            xs = x_test
            ys = np.reshape(y_test, (len(y_test),  1))  
            attributions = de.explain(func_name, target_tensor, input_tensor, xs, ys=ys)



    ss=attributions.mean(0)
    s,r=get_sr_reg(ss,M)

AttributeError: module 'tensorflow' has no attribute 'get_default_session'

In [8]:
import deeplift
from deeplift.layers import NonlinearMxtsMode
from deeplift.conversion import kerasapi_conversion as kc




revealcancel_model = kc.convert_model_from_saved_files(
                        h5_file=saved_model_file,
                        nonlinear_mxts_mode=NonlinearMxtsMode.RevealCancel)

revealcancel_func = revealcancel_model.get_target_contribs_func(find_scores_layer_idx=0, target_layer_idx=-2)


TypeError: can only concatenate str (not "KeysViewHDF5") to str

In [19]:
saved_model_file

'checkdl.h5'

In [21]:
for method_name, score_func in [
                           ('revealcancel', revealcancel_func),
                            ]:
    if method_name in need_to_run:
        print("Computing scores for:",method_name)
        scores,ranks={},{}
        scores[method_name],ranks[method_name]=[],[]

        method_to_task_to_scores = {}
        scor = np.array(score_func(
                        task_idx=0,
                        input_data_list=[x_test],
                        input_references_list=[np.zeros_like(x_test)],
                        batch_size=100,
                        progress_update=None))

        s,r=get_sr_reg(scor.mean(0),M)



NameError: name 'revealcancel_func' is not defined

In [ ]:
for method_name, score_func in [
                           ('guided_backprop_masked', guided_backprop_func_masked)
                            ]:
    if method_name in need_to_run:
        print("Computing scores for:",method_name)
        scores,ranks={},{}
        scores[method_name],ranks[method_name]=[],[]

        method_to_task_to_scores = {}
        scor = np.array(score_func(
                        task_idx=0,
                        input_data_list=[x_test],
                        input_references_list=[np.zeros_like(x_test)],
                        batch_size=100,
                        progress_update=None))

        s,r=get_sr_reg(scor.mean(0),M)

guided_backprop_model = kc.convert_model_from_saved_files(
                            h5_file=saved_model_file,
                            nonlinear_mxts_mode=NonlinearMxtsMode.GuidedBackprop)
guided_backprop_func = guided_backprop_model.get_target_multipliers_func(find_scores_layer_idx=0, target_layer_idx=-2)
guided_backprop_func_masked = lambda input_data_list, **kwargs: ((input_data_list[0]>0.0)*
                           guided_backprop_func(input_data_list=input_data_list, **kwargs))
